In [5]:
import os
from ftplib import FTP
from io import BytesIO
from datetime import datetime
import pandas as pd
import io

In [6]:
dataset = pd.read_excel('Funeral_Cover_Quotes Master File.xlsx')
dataset

,Date,user_ids,Unique Visitors
0,2024-10-14,216610000000445872505,1
1,2024-10-14,216610000000494784695,1
2,2024-10-14,216610000000494787726,1
3,2024-10-14,216610000000497736846,1
4,2024-10-14,216610000000500833764,1
...,...,...,...
30258,2025-04-21,216610000019511176719,1
30259,2025-04-21,216610000019513408953,1
30260,2025-04-21,216610000019514754914,1
30261,2025-04-21,216610000019516202570,1


In [7]:
ftp = FTP('ftp3.omniture.com')  # Replace with your FTP host

ftp.login(user='vodacomdlsvodacomfinanc_01071598', passwd='CCVUGkNi')  # Replace with your credentials

# List files and get modification times
files = ftp.nlst()

# Get modified times
file_times = []
for file in files:
    try:
        # 'MDTM' retrieves the modified time of the file (format: YYYYMMDDHHMMSS)
        modified_time = ftp.sendcmd(f'MDTM {file}')[4:].strip()
        file_times.append((file, modified_time))
    except Exception as e:
        print(f"Could not retrieve time for {file}: {e}")

# Sort by modified time (latest first)
file_times.sort(key=lambda x: x[1], reverse=True)

# Get the most recent file
latest_file = file_times[0][0]
print(f"Most recent file: {latest_file}")

# Download the latest file into memory
bio = BytesIO()
ftp.retrbinary(f'RETR {latest_file}', bio.write)
bio.seek(0)  # Reset buffer

# Assuming the file is a CSV
df = pd.read_csv(bio)

# Close FTP connection
ftp.quit()

# Show DataFrame
df.columns.values[1] = 'user_ids'
df

PermissionError: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions

In [ ]:
# Step 1: Find new user_ids (not in dataset)
new_rows = df[~df['user_ids'].isin(dataset['user_ids'])]

# Step 2: Count how many new user_ids
count_new = new_rows.shape[0]
print(f"Number of new user_ids: {count_new}")

# Step 3: Append new rows to dataset
dataset = pd.concat([dataset, new_rows], ignore_index=True)

# Step 4: Save new user_ids to a CSV file named with the week number
week_number = datetime.now().isocalendar()[1]  # Get current ISO week number
filename = f"user_ids_week_{week_number}.csv"
new_rows.to_csv(filename, index=False)

print(f"New user_ids saved to {filename}")

Number of new user_ids: 316
